<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
import pandas as pd


In [3]:
df = pd.read_csv('./WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
pd.set_option('max_columns', None)
df.describe(include='all')

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
count,7043,7043,7043.000000,7043,7043,7043.000000,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043.000000,7043,7043
unique,7043,2,NaN,2,2,NaN,2,3,3,3,3,3,3,3,3,3,2,4,NaN,6531,2
top,1113-IUJYX,Male,NaN,No,No,NaN,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,NaN,20.2,No
freq,1,3555,NaN,3641,4933,NaN,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365,NaN,11,5174
mean,NaN,NaN,0.162147,NaN,NaN,32.371149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.761692,NaN,NaN
std,NaN,NaN,0.368612,NaN,NaN,24.559481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.090047,NaN,NaN
min,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.250000,NaN,NaN
25%,NaN,NaN,0.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.500000,NaN,NaN
50%,NaN,NaN,0.000000,NaN,NaN,29.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.350000,NaN,NaN
75%,NaN,NaN,0.000000,NaN,NaN,55.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.850000,NaN,NaN


In [48]:
df['TotalCharges'].value_counts()

0          11
20.2       11
19.75       9
20.05       8
19.9        8
           ..
6428.4      1
1070.25     1
1934.45     1
2681.15     1
7030.65     1
Name: TotalCharges, Length: 6531, dtype: int64

In [47]:
pd.set_option('max_rows', 10)
df['TotalCharges'] = df['TotalCharges'].replace(' ', 0)

In [49]:
df_copy = df.drop(columns='customerID')

In [50]:
cat_cols = list(df_copy.describe(include='object'))
cat_cols.remove('TotalCharges')
cat_cols

['gender',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'Churn']

In [51]:
def str_to_float(x):
    """Takes a pd series and changes all column values to float"""
    y = float(x)
    return y

df['TotalCharges'] = df['TotalCharges'].apply(str_to_float)

In [52]:
import category_encoders as ce

In [53]:
encoder = ce.OneHotEncoder(cols=cat_cols, use_cat_names=True)

df_encoded = encoder.fit_transform(df_copy)

In [54]:
df_encoded.head()

,gender_Female,gender_Male,SeniorCitizen,Partner_Yes,Partner_No,Dependents_No,Dependents_Yes,tenure,PhoneService_No,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_No,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_Yes,OnlineSecurity_No internet service,OnlineBackup_Yes,OnlineBackup_No,OnlineBackup_No internet service,DeviceProtection_No,DeviceProtection_Yes,DeviceProtection_No internet service,TechSupport_No,TechSupport_Yes,TechSupport_No internet service,StreamingTV_No,StreamingTV_Yes,StreamingTV_No internet service,StreamingMovies_No,StreamingMovies_Yes,StreamingMovies_No internet service,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaperlessBilling_No,PaymentMethod_Electronic check,PaymentMethod_Mailed check,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),MonthlyCharges,TotalCharges,Churn_No,Churn_Yes
0,1,0,0,1,0,1,0,1,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,29.85,29.85,1,0
1,0,1,0,0,1,1,0,34,0,1,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,56.95,1889.5,1,0
2,0,1,0,0,1,1,0,2,0,1,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,53.85,108.15,0,1
3,0,1,0,0,1,1,0,45,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,42.30,1840.75,1,0
4,1,0,0,0,1,1,0,2,0,1,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,70.70,151.65,0,1


In [55]:
df_encoded.describe(include='all')

,gender_Female,gender_Male,SeniorCitizen,Partner_Yes,Partner_No,Dependents_No,Dependents_Yes,tenure,PhoneService_No,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_No,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_Yes,OnlineSecurity_No internet service,OnlineBackup_Yes,OnlineBackup_No,OnlineBackup_No internet service,DeviceProtection_No,DeviceProtection_Yes,DeviceProtection_No internet service,TechSupport_No,TechSupport_Yes,TechSupport_No internet service,StreamingTV_No,StreamingTV_Yes,StreamingTV_No internet service,StreamingMovies_No,StreamingMovies_Yes,StreamingMovies_No internet service,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaperlessBilling_No,PaymentMethod_Electronic check,PaymentMethod_Mailed check,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),MonthlyCharges,TotalCharges,Churn_No,Churn_Yes
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.00000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.00000,7043.000000,7043.000000,7043.000000,7043.0,7043.00000,7043.00000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6531.0,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN
mean,0.495244,0.504756,0.162147,0.483033,0.516967,0.700412,0.299588,32.371149,0.096834,0.903166,0.096834,0.481329,0.421837,0.343746,0.439585,0.216669,0.496663,0.286668,0.216669,0.344881,0.43845,0.216669,0.439443,0.343888,0.216669,0.493114,0.290217,0.216669,0.398978,0.384353,0.216669,0.395428,0.387903,0.216669,0.550192,0.209144,0.240664,0.592219,0.407781,0.335794,0.22888,0.219225,0.216101,64.761692,NaN,0.73463,0.26537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,18.250000,NaN,0.00000,0.00000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,35.500000,NaN,0.00000,0.00000
50%,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,29.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,70.350000,NaN,1.00000,0.00000
75%,1.000000,1.00

In [56]:
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(df_encoded)

In [57]:
target = 'Churn_Yes'

y_train = x_train[target]
y_test = x_test[target]
X_train = x_train.drop(columns=['Churn_No', target])
X_test = x_test.drop(columns=['Churn_No', target])

In [58]:
X_train.shape

(5282, 45)

In [59]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()


X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [73]:
X_train_scaled

array([[-0.98235928,  0.98235928, -0.44375466, ..., -0.53517299,
        -1.49928798, -0.70623818],
       [ 1.0179575 , -1.0179575 , -0.44375466, ..., -0.53517299,
         0.01226286,  1.00580946],
       [ 1.0179575 , -1.0179575 , -0.44375466, ..., -0.53517299,
        -1.16375691, -0.91590874],
       ...,
       [-0.98235928,  0.98235928, -0.44375466, ..., -0.53517299,
         0.83946321, -0.96542135],
       [ 1.0179575 , -1.0179575 , -0.44375466, ..., -0.53517299,
         1.71483387,  2.54205078],
       [-0.98235928,  0.98235928, -0.44375466, ..., -0.53517299,
         0.17504526,  0.78704204]])

In [71]:
import numpy as np
y_train_array = np.array(y_train)
y_test_array = np.array(y_test)

In [60]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [74]:
inputs = X_train.shape[1]
epochs = 100
batch_size = 10


model  = Sequential()
model.add(Dense(45, activation='relu', input_shape=(inputs,)))
model.add(Dense(45, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


model.fit(X_train_scaled, y_train_array,
         validation_data=(X_test_scaled, y_test_array),
         epochs=epochs,
         batch_size=batch_size
         )

Train on 5282 samples, validate on 1761 samples
Epoch 1/100
5282/5282 [==============================] - 3s 545us/sample - loss: 0.4450 - accuracy: 0.7880 - val_loss: 0.4361 - val_accuracy: 0.7933
Epoch 2/100
5282/5282 [==============================] - 1s 247us/sample - loss: 0.4178 - accuracy: 0.8014 - val_loss: 0.4315 - val_accuracy: 0.7990
Epoch 3/100
5282/5282 [==============================] - 1s 231us/sample - loss: 0.4092 - accuracy: 0.8065 - val_loss: 0.4241 - val_accuracy: 0.8052
Epoch 4/100
5282/5282 [==============================] - 1s 230us/sample - loss: 0.4032 - accuracy: 0.8124 - val_loss: 0.4314 - val_accuracy: 0.8052
Epoch 5/100
5282/5282 [==============================] - 1s 232us/sample - loss: 0.3989 - accuracy: 0.8156 - val_loss: 0.4319 - val_accuracy: 0.8052
Epoch 6/100
5282/5282 [==============================] - 1s 230us/sample - loss: 0.3951 - accuracy: 0.8137 - val_loss: 0.4301 - val_accuracy: 0.8018
Epoch 7/100
5282/5282 [==============================] - 1

In [76]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

seed = 436824
np.random.seed(seed)


def create_model():
    model = Sequential()
    model.add(Dense(45, input_dim=45, activation='relu'))
    model.add(Dense(45, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid={'batch_size': [10, 20, 40, 60, 80, 100],
           'epochs': [100]}


grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_scaled, y_train_array)


print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f'Means: {mean}, Stdev{stdev} with: {param}')

Best: 0.7608860280422585 using {'batch_size': 80, 'epochs': 100}
Means: 0.7449829666192944, Stdev0.009565032265234972 with: {'batch_size': 10, 'epochs': 100}
Means: 0.7472548342279213, Stdev0.005334676397835721 with: {'batch_size': 20, 'epochs': 100}
Means: 0.7434683907036527, Stdev0.006665379380080185 with: {'batch_size': 40, 'epochs': 100}
Means: 0.748390747477547, Stdev0.009113415082729788 with: {'batch_size': 60, 'epochs': 100}
Means: 0.7608860280422585, Stdev0.010081119675137052 with: {'batch_size': 80, 'epochs': 100}
Means: 0.7606966900062669, Stdev0.007437824160457725 with: {'batch_size': 100, 'epochs': 100}


In [81]:
param_grid={'batch_size': [80],
           'epochs': [6, 12, 18]}


grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_scaled, y_train_array)


print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f'Means: {mean}, Stdev{stdev} with: {param}')

Best: 0.8004543565859717 using {'batch_size': 80, 'epochs': 12}
Means: 0.7938281190819652, Stdev0.0036773827900083013 with: {'batch_size': 80, 'epochs': 6}
Means: 0.8004543565859717, Stdev0.005796785573926453 with: {'batch_size': 80, 'epochs': 12}
Means: 0.7925028487707078, Stdev0.00799841108012265 with: {'batch_size': 80, 'epochs': 18}


In [82]:
def create_model():
    model = Sequential()
    model.add(Dense(45, input_dim=45, activation='relu'))
    model.add(Dense(45, activation='relu'))
    model.add(Dense(45, activation='relu'))
    model.add(Dense(45, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid={'batch_size': [80],
           'epochs': [12],
           }


grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_scaled, y_train_array)


print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f'Means: {mean}, Stdev{stdev} with: {param}')

Best: 0.7900416530790044 using {'batch_size': 80, 'epochs': 12}
Means: 0.7900416530790044, Stdev0.0031735227814465063 with: {'batch_size': 80, 'epochs': 12}


In [83]:
def create_model():
    model = Sequential()
    model.add(Dense(45, input_dim=45, activation='sigmoid'))
    model.add(Dense(45, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid={'batch_size': [80],
           'epochs': [12]}


grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_scaled, y_train_array)


print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f'Means: {mean}, Stdev{stdev} with: {param}')

Best: 0.8015903078868777 using {'batch_size': 80, 'epochs': 12}
Means: 0.8015903078868777, Stdev0.0061845163181503396 with: {'batch_size': 80, 'epochs': 12}


In [87]:
def create_model():
    model = Sequential()
    model.add(Dense(45, input_dim=45, activation='sigmoid'))
    model.add(Dense(45, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='Nadam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid={'batch_size': [80],
           'epochs': [12]}


grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_scaled, y_train_array)


print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f'Means: {mean}, Stdev{stdev} with: {param}')

Best: 0.7970465727260464 using {'batch_size': 80, 'epochs': 12}
Means: 0.7970465727260464, Stdev0.009588571267075786 with: {'batch_size': 80, 'epochs': 12}


In [ ]:
def create_model():
    model = Sequential()
    model.add(Dense(45, input_dim=45, activation='sigmoid'))
    model.add(Dense(45, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid={'batch_size': [80],
           'epochs': [12]}


grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_scaled, y_train_array)


print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f'Means: {mean}, Stdev{stdev} with: {param}')

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?